#### Clean and normalize my combined header.csv from all years

In [ ]:
import numpy as np
import pandas as pd
import os

# sees all columns
pd.set_option('display.max_columns', None)

##### Finding distinct values in weight_unit columns (ex: lbs, Pounds, LB, Kgs, KG)

In [ ]:
input_path = r"..\data\bronze_raw\CBP AMS Shipping Data\raw-layer\ams_header_combined.csv"
output_path = r"..\data\silver_cleaned\ams_header_cleaned.csv"

chunk_size = 1000000

unit_counts = {} # dict to count occurrences of each unit
unit_column = "weight_unit" # this is the only column we are using in the chunk loop

In [ ]:
for chunk in pd.read_csv(input_path, usecols=[unit_column], chunksize=chunk_size):
    # Drop nulls and normalize text
    units = chunk[unit_column].dropna().astype(str).str.strip().str.lower()
    
    # Count values in this chunk
    counts = units.value_counts()
    
    for unit, count in counts.items():
        unit_counts[unit] = unit_counts.get(unit, 0) + count

kilograms          49527868
pounds              4639074
metric ton            43249
measurement ton        3114
short ton              1855
long ton                188
dtype: int64


##### All types of units for weight and how often they appear

In [18]:
# Convert to Series for easy viewing, sorted by frequency
unit_counts_series = pd.Series(unit_counts).sort_values(ascending=False)
# Show results
print(unit_counts_series)

kilograms          49527868
pounds              4639074
metric ton            43249
measurement ton        3114
short ton              1855
long ton                188
dtype: int64


#### Here we will begin turning Bronze -> Silver

In [ ]:
# converts all units to kilograms
# this will be used in our apply function later
unit_to_multiplier = {
    "kilograms": 1.0,
    "pounds": 0.453592,
    "metric ton": 1000,
    "measurement ton": 907.1847,
    "short ton": 907.1847,
    "long ton": 1016.0469
}

In [41]:
chunk_size = 1000000
first = True # writes header only for the first chunk, we do this since we are writing each chunk to a file

##### Converts weight to float type, and uses map function to convert the units to kilograms

In [ ]:
# uses my dictionary above
def map_unit_to_multiplier(u):
    if pd.isna(u):
        return np.nan
    s = str(u).strip().lower()
    # direct mapping first
    if s in unit_to_multiplier:
        return unit_to_multiplier[s]
    # common heuristics and variants
    if 'kg' in s or 'kilogram' in s:
        return 1.0
    if 'lb' in s or 'pound' in s:
        return 0.453592
    if 'metric' in s or 'tonne' in s or ' mt' in s or s == 'mt':
        return 1000.0
    if 'short' in s or ('us' in s and 'ton' in s) or (s.endswith('ton') and 'long' not in s and 'metric' not in s):
        return 907.1847
    if 'long' in s or 'imperial' in s:
        return 1016.0469
    return np.nan




for chunk in pd.read_csv(input_path, chunksize=chunk_size):

    # normalize weight unit text, makes it all lower case and strips whitespace
    chunk["weight_unit"] = chunk["weight_unit"].astype(str).str.strip().str.lower()

    # coerce weight to numeric (non-numeric -> NaN)
    chunk["weight"] = pd.to_numeric(chunk["weight"], errors='coerce')

    # compute multiplier using above function and apply
    chunk["multiplier"] = chunk["weight_unit"].apply(map_unit_to_multiplier)
    chunk["weight_kg"] = chunk["weight"] * chunk["multiplier"]

    # drop original weight and unit columns
    chunk = chunk.drop(columns=["weight", "weight_unit", "multiplier"])

    # write/appends to silver CSV
    chunk.to_csv(
        output_path,
        # this part is good practice, write if, then append
        mode="w" if first else "a",
        index=False,
        header=first
    )

    first = False

C:\Users\stanl\AppData\Local\Temp\ipykernel_34320\2558675388.py:25: DtypeWarning: Columns (17,18,25,26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(input_path, chunksize=chunk_size):
C:\Users\stanl\AppData\Local\Temp\ipykernel_34320\2558675388.py:25: DtypeWarning: Columns (26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(input_path, chunksize=chunk_size):
C:\Users\stanl\AppData\Local\Temp\ipykernel_34320\2558675388.py:25: DtypeWarning: Columns (25,26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(input_path, chunksize=chunk_size):
C:\Users\stanl\AppData\Local\Temp\ipykernel_34320\2558675388.py:25: DtypeWarning: Columns (26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(input_path, chunksize=chunk_size):
C:\Users\stanl\A

In [45]:
silver_sample = pd.read_csv(r"..\data\silver_cleaned\ams_header_cleaned.csv", nrows=1000)
silver_sample.head(20)

np.float64(0.037)

In [ ]:
silver_sample.tail(5)
# silver_sample["measurement_unit"].notna().mean()

,identifier,carrier_code,vessel_country_code,vessel_name,port_of_unlading,estimated_arrival_date,foreign_port_of_lading_qualifier,foreign_port_of_lading,manifest_quantity,manifest_unit,measurement,measurement_unit,record_status_indicator,place_of_receipt,port_of_destination,foreign_port_of_destination_qualifier,foreign_port_of_destination,conveyance_id_qualifier,conveyance_id,in_bond_entry_type,mode_of_transportation,secondary_notify_party_1,secondary_notify_party_2,secondary_notify_party_3,secondary_notify_party_4,secondary_notify_party_5,secondary_notify_party_6,secondary_notify_party_7,secondary_notify_party_8,secondary_notify_party_9,secondary_notify_party_10,actual_arrival_date,source_year,weight_kg
999,20180101999,DMAL,HK,OOCL BERLIN,"Charleston, South Carolina",2017-12-23,Schedule K Foreign Port,"Yantian,China (Mainland)",1139,CTN,0,NaN,New,YANTIAN,NaN,NaN,NaN,IMO Number/Lloyds Number,9622605,NaN,"Vessel, containerized",COSU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-12-30,2018,9530.0


#### Drop unneeded columns

In [58]:
drop_columns = [
    # quantity and units of hazardous materials, not used for my analysis
    "manifest_quantity",
    "manifest_unit",
    
    # the volume of the cargo
    "measurement",
    "measurement_unit",
    
    # not needed since we have vessel_code (coutnry) and port of unlading (US ports)
    "foreign_port_of_lading_qualifer",
    "foreign_port_of_destination_qualifier",
    "foreign_port_of_destination",
    "port_of_destination",
    "place_of_receipt",
    
    
    # extremely sparse and not used
    "in_bond_entry_type",
    
    # notifcations, a lot are empty
    "secondary_notify_party_1",
    "secondary_notify_party_2",
    "secondary_notify_party_3",
    "secondary_notify_party_4",
    "secondary_notify_party_5",
    "secondary_notify_party_6",
    "secondary_notify_party_7",
    "secondary_notify_party_8",
    "secondary_notify_party_9",
    "secondary_notify_party_10",
]

##### Temp file that will replace the old one

In [ ]:
file_path = r"..\data\silver_cleaned\ams_header_cleaned.csv"
temp_path = r"..\data\silver_cleaned\silver_header_temp.csv"
chunk_size = 1000000

In [ ]:
# Process in chunks
first_chunk = True
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    # Keep only existing columns to avoid errors
    existing_drop_columns = [c for c in drop_columns if c in chunk.columns]
    
    # Drop the unwanted columns
    chunk = chunk.drop(columns=existing_drop_columns)
    
    # Write to temporary file
    chunk.to_csv(
        temp_path,
        mode="w" if first_chunk else "a",
        index=False,
        header=first_chunk
    )
    
    first_chunk = False

os.replace(temp_path, file_path)
print(f"Columns dropped and file overwritten: {file_path}")

C:\Users\stanl\AppData\Local\Temp\ipykernel_34320\2798508519.py:2: DtypeWarning: Columns (15,16,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\stanl\AppData\Local\Temp\ipykernel_34320\2798508519.py:2: DtypeWarning: Columns (24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\stanl\AppData\Local\Temp\ipykernel_34320\2798508519.py:2: DtypeWarning: Columns (23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\stanl\AppData\Local\Temp\ipykernel_34320\2798508519.py:2: DtypeWarning: Columns (24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\stanl\AppData\L

Columns dropped and file overwritten: ..\data\silver_cleaned\ams_header_cleaned.csv


In [62]:
silver_sample = pd.read_csv(r"..\data\silver_cleaned\ams_header_cleaned.csv", nrows=100)
silver_sample.head(5)

,identifier,carrier_code,vessel_country_code,vessel_name,port_of_unlading,estimated_arrival_date,foreign_port_of_lading_qualifier,foreign_port_of_lading,record_status_indicator,conveyance_id_qualifier,conveyance_id,mode_of_transportation,actual_arrival_date,source_year,weight_kg
0,201801010,DFDS,GB,EVER SIGMA,"Los Angeles, California",2017-02-14,Schedule K Foreign Port,"Kaohsiung,China (Taiwan)",New,IMO Number/Lloyds Number,9300398,"Vessel, containerized",2017-02-15,2018,183.0
1,201801011,DFDS,GB,EVER SIGMA,"Los Angeles, California",2017-02-14,Schedule K Foreign Port,"Kaohsiung,China (Taiwan)",New,IMO Number/Lloyds Number,9300398,"Vessel, containerized",2017-02-15,2018,1096.0
2,201801012,DFDS,GB,EVER SIGMA,"Los Angeles, California",2017-02-14,Schedule K Foreign Port,"Kaohsiung,China (Taiwan)",New,IMO Number/Lloyds Number,9300398,"Vessel, containerized",2017-02-15,2018,758.0
3,201801013,DFDS,GB,EVER SIGMA,"Los Angeles, California",2017-02-14,Schedule K Foreign Port,"Kaohsiung,China (Taiwan)",New,IMO Number/Lloyds Number,9300398,"Vessel, containerized",2017-02-15,2018,2050.0
4,201801014,DFDS,GB,EVER SIGMA,"Los Angeles, California",2017-02-14,Schedule K Foreign Port,"Kaohsiung,China (Taiwan)",New,IMO Number/Lloyds Number,9300398,"Vessel, containerized",2017-02-15,2018,149.0


#### Reorder columns
##### Found out it doesn't matter when we move data to SQL server

In [68]:
file_path = r"..\data\silver_cleaned\ams_header_cleaned.csv"
temp_path = r"..\data\silver_cleaned\ams_header_silver.csv"

column_order = [
    "identifier",
    "conveyance_id_qualifier",
    "conveyance_id",
    "carrier_code",
    "mode_of_transportation",
    "vessel_name",
    "vessel_country_code",
    "port_of_unlading",
    "foreign_port_of_lading",
    "estimated_arrival_date",
    "actual_arrival_date",
    "source_year",
    "weight_kg",
    "record_status_indicator"
]

first_chunk = True

In [69]:
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    # Keep only columns that exist in this chunk
    existing_columns = [c for c in column_order if c in chunk.columns]
    
    # Reorder
    chunk = chunk[existing_columns]
    
    # Write to temporary file
    chunk.to_csv(
        temp_path,
        mode="w" if first_chunk else "a",
        index=False,
        header=first_chunk
    )
    first_chunk = False

# Replace original file with reordered file
os.replace(temp_path, file_path)
print(f"Columns reordered and file overwritten: {file_path}")

Columns reordered and file overwritten: ..\data\silver_cleaned\ams_header_cleaned.csv


In [ ]:
final_sample = pd.read_csv(r"..\data\silver_cleaned\ams_header_cleaned.csv", nrows=100)
final_sample.tail()

,identifier,conveyance_id_qualifier,conveyance_id,carrier_code,mode_of_transportation,vessel_name,vessel_country_code,port_of_unlading,foreign_port_of_lading,estimated_arrival_date,actual_arrival_date,source_year,weight_kg,record_status_indicator
0,201801010,IMO Number/Lloyds Number,9300398,DFDS,"Vessel, containerized",EVER SIGMA,GB,"Los Angeles, California","Kaohsiung,China (Taiwan)",2017-02-14,2017-02-15,2018,183.0,New
1,201801011,IMO Number/Lloyds Number,9300398,DFDS,"Vessel, containerized",EVER SIGMA,GB,"Los Angeles, California","Kaohsiung,China (Taiwan)",2017-02-14,2017-02-15,2018,1096.0,New
2,201801012,IMO Number/Lloyds Number,9300398,DFDS,"Vessel, containerized",EVER SIGMA,GB,"Los Angeles, California","Kaohsiung,China (Taiwan)",2017-02-14,2017-02-15,2018,758.0,New
3,201801013,IMO Number/Lloyds Number,9300398,DFDS,"Vessel, containerized",EVER SIGMA,GB,"Los Angeles, California","Kaohsiung,China (Taiwan)",2017-02-14,2017-02-15,2018,2050.0,New
4,201801014,IMO Number/Lloyds Number,9300398,DFDS,"Vessel, containerized",EVER SIGMA,GB,"Los Angeles, California","Kaohsiung,China (Taiwan)",2017-02-14,2017-02-15,2018,149.0,New
